## Getting Started

### Prerequisites
1. Create an Ollama account at [ollama.com](https://ollama.com)
2. Generate an API key from [ollama.com/settings/keys](https://ollama.com/settings/keys)
3. Set the API key in the `.env` file in this folder

### Installation
```bash
pip install ollama python-dotenv requests
```

In [1]:
# Setup: Install required packages and import libraries
import sys
import subprocess

packages = ['ollama', 'python-dotenv', 'requests']

for package in packages:
    try:
        __import__(package.replace('-', '_'))
    except ImportError:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

print("✓ All packages installed successfully!")

Installing python-dotenv...
✓ All packages installed successfully!


In [2]:
# Import all required libraries
import os
from pathlib import Path
from dotenv import load_dotenv
import ollama
from typing import Dict


# Load environment variables from .env file
# Use Path.cwd() since __file__ is not defined in Jupyter notebooks
env_path = Path.cwd() / '.env'
if env_path.exists():
    load_dotenv(env_path)
else:
    # Try to find .env in the ollama-cloud directory
    load_dotenv(dotenv_path='d:\\ollama-n8n\\ollama-cloud\\.env')

# Get API key and configuration
OLLAMA_API_KEY = os.getenv('OLLAMA_API_KEY')
OLLAMA_ENDPOINT = os.getenv('OLLAMA_API_ENDPOINT', 'https://ollama.com')
DEFAULT_MODEL = os.getenv('DEFAULT_MODEL', 'gpt-oss:120b')

# Verify API key is set
if not OLLAMA_API_KEY or OLLAMA_API_KEY == 'your_api_key_here':
    print("⚠️  WARNING: OLLAMA_API_KEY not set in .env file!")
    print("Please set your API key in the .env file: https://ollama.com/settings/keys")
else:
    print(f"✓ Ollama API configured")
    print(f"  Endpoint: {OLLAMA_ENDPOINT}")
    print(f"  Default Model: {DEFAULT_MODEL}")

# Initialize Ollama client for cloud API
client = ollama.Client(
    host=OLLAMA_ENDPOINT,
    headers={'Authorization': f'Bearer {OLLAMA_API_KEY}'}
)

✓ Ollama API configured
  Endpoint: https://ollama.com
  Default Model: gpt-oss:120b


## 3. Thinking Capability - Extended Reasoning

Some models (Qwen3-next, DeepSeek-R1, GPT-OSS) support the "thinking" feature where models show their reasoning process before answering. This is valuable for:
- Auditing model reasoning
- Understanding decision-making
- Detecting hallucinations
- Solving complex problems requiring step-by-step reasoning

In [3]:
def test_thinking_with_streaming(model: str = 'qwen3-next:80b', prompt: str = "How many times does the letter 'r' appear in 'strawberry'?") -> Dict[str, str]:
    """
    Test thinking capability with streaming.
    Shows both the reasoning trace and final answer.
    
    Note: GPT-OSS requires think='low'/'medium'/'high' instead of True/False
    
    Args:
        model: Model with thinking support (qwen3-next:80b, deepseek-r1, gpt-oss:120b)
        prompt: The problem to think about
    
    Returns:
        Dictionary with thinking and content
    """
    print(f"Testing thinking capability on {model}...")
    print(f"   Prompt: {prompt}\n")
    
    # Determine think parameter based on model
    think_param = 'low' if 'gpt-oss' in model else True
    
    thinking = ""
    content = ""
    in_thinking = False
    
    stream = client.chat(
        model=model,
        messages=[{'role': 'user', 'content': prompt}],
        think=think_param,
        stream=True
    )
    
    for chunk in stream:
        if chunk.message.thinking:
            if not in_thinking:
                print("Thinking Process:")
                print("-" * 50)
                in_thinking = True
            print(chunk.message.thinking, end='', flush=True)
            thinking += chunk.message.thinking
        elif chunk.message.content:
            if in_thinking:
                print("\n" + "-" * 50)
                print("\nFinal Answer:")
                print("-" * 50)
                in_thinking = False
            print(chunk.message.content, end='', flush=True)
            content += chunk.message.content
    
    print("\n" + "-" * 50 + "\n")
    return {"thinking": thinking, "content": content}

# Test thinking with Qwen3 (supports thinking)
print("Testing with Qwen3 (thinking-enabled model):\n")
result = test_thinking_with_streaming(model='qwen3-next:80b')

Testing with Qwen3 (thinking-enabled model):

Testing thinking capability on qwen3-next:80b...
   Prompt: How many times does the letter 'r' appear in 'strawberry'?

Thinking Process:
--------------------------------------------------
First, the question is: "How many times does the letter 'r' appear in 'strawberry'?"

I need to count the occurrences of the letter 'r' in the word "strawberry".

Let me write down the word: s-t-r-a-w-b-e-r-r-y.

Now, I'll go through each letter one by one and count how many times 'r' appears.

Starting with the first letter: 's' – not 'r'.

Second letter: 't' – not 'r'.

Third letter: 'r' – yes, that's one 'r'.

Fourth letter: 'a' – not 'r'.

Fifth letter: 'w' – not 'r'.

Sixth letter: 'b' – not 'r'.

Seventh letter: 'e' – not 'r'.

Eighth letter: 'r' – yes, that's two 'r's.

Ninth letter: 'r' – yes, that's three 'r's.

Tenth letter: 'y' – not 'r'.

I think I have it: positions 3, 8, and 9 are 'r's.

Let me spell it out: s-t-r-a-w-b-e-r-r-y.

- Index 1: 